In [ ]:
pip install deepface flask pillow flask-cors numpy opencv-python


In [ ]:
!pip install flask pyngrok spotipy


In [ ]:
!ngrok authtoken 2uupJNTPLof8UaeQ8aTj8gWAfBY_2PVV6ciL14KWDQqcrxVVh

In [7]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from deepface import DeepFace
import os
from PIL import Image
import numpy as np
from io import BytesIO
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

app = Flask(__name__)

# Function to convert numpy types to regular Python types
def convert_numpy_types(data):
    if isinstance(data, np.generic):
        return data.item()
    elif isinstance(data, dict):
        return {key: convert_numpy_types(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_numpy_types(item) for item in data]
    return data

# Spotify Auth
def authenticate_spotify():
    client_credentials_manager = SpotifyClientCredentials(
        client_id="2fe8578c9fe74f40b664f04d56eb9e96",
        client_secret="35b41a84fa564b3f88839114661b3029"
    )
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    return sp

# Search Spotify track
def search_spotify_track(features):
    sp = authenticate_spotify()
    query = f"valence:{features['valence']} energy:{features['energy']} danceability:{features['danceability']} tempo:{features['tempo']}"
    results = sp.search(q=query, type="track", limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return {
            "song_name": track['name'],
            "song_id": track['id'],
            "artist": track['artists'][0]['name']
        }
    else:
        return {"error": "No track found matching these features"}

# Audio feature recommendation logic
def compute_audio_features_and_recommend(emotion_scores):
    good_emotions = ["happy", "surprise", "neutral"]
    total = sum(emotion_scores.values())
    normalized = {k: v / total for k, v in emotion_scores.items()}
    max_emotion_val = max(normalized.values())
    if max_emotion_val < 0.6:
        for emo in good_emotions:
            if emo in normalized:
                normalized[emo] *= 1.2

    feature_map = {
        "angry": {"valence": 0.3, "energy": 0.9, "danceability": 0.6, "tempo": 130},
        "disgust": {"valence": 0.2, "energy": 0.5, "danceability": 0.3, "tempo": 110},
        "fear": {"valence": 0.2, "energy": 0.8, "danceability": 0.4, "tempo": 120},
        "happy": {"valence": 0.9, "energy": 0.9, "danceability": 0.8, "tempo": 128},
        "neutral": {"valence": 0.5, "energy": 0.5, "danceability": 0.5, "tempo": 110},
        "sad": {"valence": 0.1, "energy": 0.3, "danceability": 0.3, "tempo": 90},
        "surprise": {"valence": 0.8, "energy": 0.8, "danceability": 0.7, "tempo": 125}
    }

    blended = {"valence": 0.0, "energy": 0.0, "danceability": 0.0, "tempo": 0.0}
    for emotion, weight in normalized.items():
        for feature in blended:
            blended[feature] += feature_map[emotion][feature] * weight

    return search_spotify_track(blended)

# Main route
@app.route('/detect_emotion', methods=['POST'])
def detect_emotion():
    try:
        file = request.files['image']
        img = Image.open(file.stream)
        temp_image_path = "/tmp/temp_image.jpg"
        img = img.convert("RGB")
        img.save(temp_image_path)

        analysis = DeepFace.analyze(temp_image_path, actions=['emotion'])
        analysis = convert_numpy_types(analysis)

        emotion_scores = analysis[0]['emotion']
        recommendation = compute_audio_features_and_recommend(emotion_scores)

        return jsonify({
            'dominant_emotion': analysis[0]['dominant_emotion'],
            'emotion_details': emotion_scores,
            'recommended_song': recommendation
        })
    except Exception as e:
        return jsonify({'error': str(e)})

# Ngrok expose
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(port=5000)


Public URL: NgrokTunnel: "https://982e-34-125-50-184.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 21:10:14] "POST /detect_emotion HTTP/1.1" 200 -
